In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer
import numpy as np
import tensorflow as tf
import tflearn
import random

In [2]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 3.2MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fb/06/72/0478c938ca315c6fddcce8233b80ec91a115ce4496a27e8c90
Successfully built tflearn


In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import pickle
data=pickle.load(open('drive/Colab Notebooks/chatbot/training_data','rb'))

In [0]:
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [0]:
import json
with open('drive/Colab Notebooks/chatbot/intents.json') as json_data:
    intents = json.load(json_data)

In [11]:

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# load our saved model
model.load('drive/Colab Notebooks/chatbot/model.tflearn')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from /content/drive/Colab Notebooks/chatbot/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer().stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
error_thresh=0.25
def classify(sentence):
     # generate probabilities from the model
    results=model.predict([bow(sentence,words)])[0]
     # filter out predictions below a threshold
    results=[[i,r] for i,r in enumerate(results) if r>error_thresh]
     # sort by strength of probability
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
         # return tuple of intent and probability
    return return_list

In [0]:
context={}
def response(sentence,user_id='123',show_details=False):
    results=classify(sentence)
     # if we have a classification then find the matching intent tag
    if results:
         # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                 # find a tag matching the first result
                    if i['tag'] == results[0][0]:
                         # set context for this intent if necessary
                        if 'context_set' in i:
                            if show_details: print('context',i['context_set'])
                            context[user_id]=i['context_set']
                            
                        # check if this intent is contextual and applies to this user's conversation
                        if not 'context_filter' in i or (user_id in context and 'context_filter' in i and i['context_filter']==context[user_id]):
                            if show_details: print('tag:', i['tag'])
                            # a random response from the intent
                            return random.choice(i['responses'])
            results.pop(0)

In [22]:
response('tell me about wheat protection')


'SPRAY OF ZINK SULPHATE 5 GRAM +UREA 10 GRAM PER LITER WATER'